In [ ]:
from ckks.ckks_parameters import CKKSParameters
from ckks.ckks_key_generator import CKKSKeyGenerator
from ckks.ckks_encoder import CKKSEncoder

# Test 1: Encode and Decode - Verifying Inverse Relationship

In [12]:
# Test setup
poly_degree = 4
ciph_modulus = 1 << 600
q0 = 2**24
big_modulus = 1 << 1200 # Used for bootstrapping
scaling_factor = 2**7-1
ciph_modulus = scaling_factor**2 * q0  # ql
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor)
message = [4.0 + 0j, 3 + 0j]
print("Message:", message)

key_generator = CKKSKeyGenerator(params)
public_key = key_generator.public_key
secret_key = key_generator.secret_key

encoder = CKKSEncoder(params) # Contains encode and decode functions
poly = encoder.encode(message, params.scaling_factor)
print("Encoded polynomial:", str(poly))
decoded_message = encoder.decode(poly)
print("Decoded polynomial:", decoded_message)

assert len(decoded_message) == len(message), "Decoded polynomial has incorrect length"
print("Encode and decode are inverse:", np.allclose(decoded_message, message, atol=1e-1))

Message: [(4+0j), (3+0j)]
Encoded polynomial: -44x^3 + 45x + 445
Decoded polynomial: [(3.9994685316976595+0.005567769930602762j), (3.008405484050372-0.005567769930602762j)]
Encode and decode are inverse: True


# Test 2: Addition Is Homomorphic

In [31]:
# Test setup
from util.plaintext import Plaintext

poly_degree = 2048
ciph_modulus = 1 << 40
big_modulus = 1 << 1200 # Used for bootstrapping
scaling_factor = 1 << 30
# poly_degree = 4
# ciph_modulus = 1 << 600
# q0 = 2**24
# big_modulus = 1 << 1200 # Used for bootstrapping
# scaling_factor = 2**7-1
# ciph_modulus = scaling_factor**2 * q0  # ql
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor)
message1 = [4.0 + 0j, 3 + 0j]
print("Message1:", message)
message2 = [4.0 + 0j, 3 + 0j]
print("Message2:", message)

key_generator = CKKSKeyGenerator(params)
public_key = key_generator.public_key
secret_key = key_generator.secret_key

encoder = CKKSEncoder(params) # Contains encode and decode functions
poly1 = encoder.encode(message1, params.scaling_factor)
poly2 = encoder.encode(message2, params.scaling_factor)
poly_sum = poly1.poly.add(poly2.poly)
# poly_sum = poly1.poly.add(poly2.poly, params.ciph_modulus) # NOTE: For some reason this doesn't work
plain_poly_sum = Plaintext(poly_sum, params.scaling_factor)

print("Encoded polynomial1:", str(poly1))
print("Encoded polynomial2:", str(poly2))
print("Encoded polynomial sum:", str(poly_sum))

decoded_message_sum = encoder.decode(plain_poly_sum)
true_message_sum = [m1 + m2 for m1, m2 in zip(message1, message2)]
print("True polynomial sum:", true_message_sum)
print("Decoded polynomial sum:", decoded_message_sum)

assert len(decoded_message_sum) == len(message), "Decoded polynomial has incorrect length"
print("Addition is homomorphic:", np.allclose(decoded_message_sum, true_message_sum, atol=1e-1))


Message1: [(4+0j), (3+0j)]
Message2: [(4+0j), (3+0j)]
Encoded polynomial1: -379625061x^3 + 379625062x + 3758096384
Encoded polynomial2: -379625061x^3 + 379625062x + 3758096384
Encoded polynomial sum: -759250122x^3 + 759250124x + 7516192768
True polynomial sum: [(8+0j), (6+0j)]
Decoded polynomial sum: [(7.999999997373708+1.3170890489355713e-09j), (6.000000002626292-1.3170890489355713e-09j)]
Addition is homomorphic: True


# Test 3: Multiplication Is Homomorphic

In [26]:
# Test setup
from util.plaintext import Plaintext

poly_degree = 2048
ciph_modulus = 1 << 40
big_modulus = 1 << 1200 # Used for bootstrapping
scaling_factor = 1 << 30
# ciph_modulus = scaling_factor**2 * q0  # ql
# poly_degree = 4
# ciph_modulus = 1 << 600
# q0 = 2**24
# big_modulus = 1 << 1200 # Used for bootstrapping
# scaling_factor = 2**7-1
# ciph_modulus = scaling_factor**2 * q0  # ql
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor)
message1 = [4.0 + 0j, 3 + 0j]
print("Message1:", message)
message2 = [4.0 + 0j, 3 + 0j]
print("Message2:", message)

key_generator = CKKSKeyGenerator(params)
public_key = key_generator.public_key
secret_key = key_generator.secret_key

encoder = CKKSEncoder(params) # Contains encode and decode functions
poly1 = encoder.encode(message1, params.scaling_factor)
poly2 = encoder.encode(message2, params.scaling_factor)
poly_prod = poly1.poly.multiply_naive(poly2.poly) # Use naive multiplication without NTT or CRT
plain_poly_prod = Plaintext(poly_prod, params.scaling_factor**2)

print("Encoded polynomial1:", str(poly1))
print("Encoded polynomial2:", str(poly2))
print("Encoded polynomial prod:", str(poly_prod))

decoded_message_prod = encoder.decode(plain_poly_prod)
true_message_product = [m1 * m2 for m1, m2 in zip(message1, message2)]
print("True polynomial prod:", true_message_product)
print("Decoded polynomial prod:", decoded_message_prod)

assert len(decoded_message_prod) == len(message), "Decoded polynomial has incorrect length"
print("Multiplication is homomorphic:", np.allclose(decoded_message_prod, true_message_product, atol=1e-1))


Message1: [(4+0j), (3+0j)]
Message2: [(4+0j), (3+0j)]
Encoded polynomial1: -379625061x^3 + 379625062x + 3758096384
Encoded polynomial2: -379625061x^3 + 379625062x + 3758096384
Encoded polynomial prod: -2853335138039758848x^3 + 759250123x^2 + 2853335145555951616x + 14411518806071633020
True polynomial prod: [(16+0j), (9+0j)]
Decoded polynomial prod: [(15.999999989494833+5.268356149772113e-09j), (9.000000007878876-3.951267137265735e-09j)]
Multiplication is homomorphic: True
